In [1]:
# Import 
 
import numpy as np
import matplotlib.pyplot as plt
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env import *
import random
 
size_board = 4
 
 
env = Chess_Env(size_board)
S, X, allowed_a = env.Initialise_game()
N_a = np.shape(allowed_a)[0]
N_in = np.shape(X)[0]
N_h = 200
 
import numpy as np
 
epsilon_0 = 0.2     # STARTING VALUE OF EPSILON FOR THE EPSILON-GREEDY POLICY
beta = 0.00005      # THE PARAMETER SETS HOW QUICKLY THE VALUE OF EPSILON IS DECAYING (SEE epsilon_f BELOW)
gamma = 0.85        # THE DISCOUNT FACTOR
eta = 0.0035        # THE LEARNING RATE
N_Games = 1000 
 
env = Chess_Env(size_board)
 
N_a = np.shape(allowed_a)[0]   # TOTAL NUMBER OF POSSIBLE ACTIONS
N_in = np.shape(X)[0]    ## INPUT SIZE
N_h = 200   
 
w1 = np.random.randn(N_in, N_h)
b1 = np.zeros((1, N_h))
w2 = np.random.randn(N_h, N_a)
b2 = np.zeros((1, N_a))
 
dW1 = np.zeros(w1.shape)
dW2 = np.zeros(w2.shape)
 
dbias_W1 = np.zeros(b1.shape)
dbias_W2 = np.zeros(b2.shape)
 
def f_prop(X, w1, b1, w2, b2, allowed_a):
    # First layer
    hidden = np.dot(X, w1) + b1
    hidden_out = np.maximum(hidden, 0)  # ReLU activation function
 
    # Second layer
    output = np.dot(hidden_out, w2) + b2
    output *= allowed_a.T
    output = np.maximum(output, 0)
    action_index = np.argmax(output)  # Argmax activation function
    return action_index, output[0], hidden_out
 
def back_prop(X, hidden_out, w1, b1, w2, b2, Q_current_value, Q_current_index):
    dW1 = np.zeros(w1.shape)
    dW2 = np.zeros(w2.shape)
    dbias_W1 = np.zeros(b1.shape)
    dbias_W2 = np.zeros(b2.shape)
 
    S_Next, X_Next, Mask, Reward, Done = env.OneStep(Q_current_index)
    print(X_Next)
    print(Done)
    if Done == 1:
        Q_Target = Reward - Q_current_value
    else:
        Q_next_index, All_Q_values_next, _ = f_prop(X_Next, w1, b1, w2, b2, Mask)    
        Q_Target = Reward + gamma * All_Q_values_next[Q_next_index] - Q_current_value
 
    e_n = Q_Target - Q_current_value
 
    # Backpropagation: output layer -> hidden layer
    delta2 = np.dot(w2, 0) * e_n
    #print("delta ", delta2.shape)
    #print(hidden_out.shape)
 
    #print(np.outer(hidden_out, delta2.T).shape)
 
 
    #reshaped_product = np.dot(delta2.T, hidden.T)
    #dW2 += reshaped_product   
    dW2 += np.outer(hidden_out, delta2)
    print(delta2.shape)
    dbias_W2 += delta2
 
    # Backpropagation: hidden layer -> input layer
    delta1 = np.heaviside(w1, 0) * np.dot(w2.T, delta2)
    dW1 += np.outer(X, delta1)
    dbias_W1 += delta1
 
    w1 -= eta * dW1
    b1 -= eta * dbias_W1
    w2 -= eta * dW2
    b2 -= eta * dbias_W2
 
    return w1, b1, w2, b2
 
i = 0
 
while(N_Games > i):
    Done = 0
    S, X, allowed_a = env.Initialise_game() 
    epsilon_f = epsilon_0 / (1 + beta * i)
 
    while not Done:
        # Exploration vs Exploitation
        if np.random.uniform(0, 1) < epsilon_f:
            # Exploration: randomly choose an action
            Q_current_index = np.random.choice(np.arange(N_a))
        else:
            # Exploitation: choose the action with max value (greedy)
            Q_current_index, _, _ = f_prop(X, w1, b1, w2, b2, allowed_a)
 
        _, All_Q_values, Current_hidden = f_prop(X, w1, b1, w2, b2, allowed_a)
        Q_current = All_Q_values[Q_current_index]
        w1, b1, w2, b2 =  back_prop(X, Current_hidden, w1, b1, w2, b2, Q_current, Q_current_index)
 
        S, X, allowed_a, _, Done = env.OneStep(Q_current_index)
    i +=1

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
0


ValueError: operands could not be broadcast together with shapes (200,32) (200,6400) (200,32) 